In [21]:
from __future__ import division
import numpy as np
import scipy.io as scipy
import matplotlib.pyplot as plt
!pip install mne
import mne
from numpy import *
from numpy.fft import *
import scipy.signal as signal
from matplotlib.pyplot import *
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
import sklearn

In [22]:
def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    def _check_keys(d):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in d:
            if isinstance(d[key], scipy.matlab.mio5_params.mat_struct):
                d[key] = _todict(d[key])
        return d

    def _todict(matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, scipy.matlab.mio5_params.mat_struct):
                d[strg] = _todict(elem)
            elif isinstance(elem, np.ndarray):
                d[strg] = _tolist(elem)
            else:
                d[strg] = elem
        return d

    def _tolist(ndarray):
        '''
        A recursive function which constructs lists from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        '''
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, scipy.matlab.mio5_params.mat_struct):
                elem_list.append(_todict(sub_elem))
            elif isinstance(sub_elem, np.ndarray):
                elem_list.append(_tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list
    data = scipy.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

In [26]:
p2_all = loadmat('P2_AllLifts.mat')
print( "All Lifts loaded")



#these are the series we are looking at for subject 2. Everything except series 1 and 6
trials_included=[2, 3, 4, 5, 7, 8, 9] 

#corresponding row indices in P.AllLifts where each series starts. Index is off by one bc of how python indexes things
trialStartIdx = [28, 62, 96, 130, 192, 226, 260]

#data structure to hold all EEG trials 
eeg_grasp_closed = []
eeg_grasp_open = []
emg_grasp_closed = []
emg_grasp_open = []
series_counter = 0
trial_counter = 1
i = 0
for s in trials_included:
    # load all the series specified in trials_included
    series_filename = 'WS_P2_S{series}.mat'.format(series = s)
    p_series = loadmat(series_filename)
    print("Series data loaded")
    series_counter = series_counter + 1
    print("series_counter: ", series_counter)

    #identifying the starting row index in p.ALLLifts for the first trial of the current series
    tStart = trialStartIdx[i]
    i += 1
    trial_counter = 1
    #range for the for loop is 34 values bc there are 34 trials in each series
    for trial_num in range(tStart, tStart + 34):
        
        #retrieving the point in time when both fingers touched object
        #index values are one cloumn and one row before target value
        tBothDigitTouch = np.array(p2_all['P']['AllLifts'])[trial_num, 14]
        
        tBothReleased = np.array(p2_all['P']['AllLifts'])[trial_num, 22]
        
        #converting time points to row indices to be accessed in windowed struct
        #500 samples every second, so timepoint should be divided by 0.002
        idxCloseEnd = int(tBothDigitTouch // 0.002) + int(0.8//0.002)
        idxCloseStart = int(tBothDigitTouch // 0.002)- int(0.8//0.002)
        
        idxOpenEnd = int(tBothReleased // 0.002)+ int(0.8//0.002)
        idxOpenStart = int(tBothReleased // 0.002)- int(0.8//0.002)
        
        idxCloseEndEMG = int(tBothDigitTouch // 0.0002) + int(0.8//0.0002)
        idxCloseStartEMG = int(tBothDigitTouch // 0.0002) - int(0.8//0.0002)
        
        idxOpenEndEMG = int(tBothReleased // 0.0002) + int(0.8//0.0002)
        idxOpenStartEMG = int(tBothReleased // 0.0002) - int(0.8//0.0002)

        eeg_trial = np.array(p_series['ws']['win'][0]['eeg'])[idxCloseStart:idxCloseEnd].T 
        eeg_grasp_closed.append(eeg_trial)
        
        eeg_trial = np.array(p_series['ws']['win'][0]['eeg'])[idxOpenStart:idxOpenEnd].T 
        eeg_grasp_open.append(eeg_trial)
        
        emg_trial = np.array(p_series['ws']['win'][0]['emg'])[idxCloseStartEMG:idxCloseEndEMG].T 
        emg_grasp_closed.append(emg_trial)
        
        emg_trial = np.array(p_series['ws']['win'][0]['emg'])[idxOpenStartEMG:idxOpenEndEMG].T 
        emg_grasp_open.append(emg_trial)
        #print("trial counter: ", trial_counter)
        trial_counter = trial_counter +1
                                                            

All Lifts loaded
Series data loaded
series_counter:  1
Series data loaded
series_counter:  2
Series data loaded
series_counter:  3
Series data loaded
series_counter:  4
Series data loaded
series_counter:  5
Series data loaded
series_counter:  6
Series data loaded
series_counter:  7


In [27]:
series = [2, 3, 4, 5, 7, 8, 9]
eeg_close = {}
counter = 0
for i in range (0, len(series)):
    new_list  = []
    for s in range(34*(i-1), 34*i):
        new_list.append(eeg_grasp_closed[s])
    eeg_close[series[i]] = new_list
print(eeg_close.keys())
print(eeg_close.values())


dict_keys([2, 3, 4, 5, 7, 8, 9])
dict_values([[array([[-485., -487., -491., ...,   55.,   71.,   52.],
       [-220., -203., -203., ...,  271.,  276.,  255.],
       [ 290.,  267.,  252., ...,  276.,  276.,  228.],
       ...,
       [ 509.,  557.,  531., ...,   45.,   24.,   48.],
       [ 458.,  512.,  454., ...,   86.,   73.,   79.],
       [ 314.,  382.,  335., ...,  183.,  158.,  139.]], dtype=float32), array([[-138., -146., -166., ..., -100.,  -74.,  -22.],
       [ 125.,  134.,   93., ...,  318.,  310.,  339.],
       [ 282.,  305.,  282., ...,  209.,  197.,  222.],
       ...,
       [ 226.,  150.,  103., ...,   13.,   -9.,   61.],
       [ 334.,  197.,  124., ...,   28.,   16.,   87.],
       [ 539.,  381.,  198., ...,  176.,   88.,  182.]], dtype=float32), array([[-522., -527., -556., ...,  -31.,  -20.,  -43.],
       [-253., -270., -264., ...,  273.,  262.,  228.],
       [ 197.,  163.,  153., ...,  152.,  169.,  216.],
       ...,
       [ 370.,  383.,  365., ...,   50.,   

In [34]:
series = [2, 3, 4, 5, 7, 8, 9]
eeg_open = {}
counter = 0
for i in range (0, len(series)):
    new_list  = []
    for s in range(34*(i-1), 34*i):
        new_list.append(eeg_grasp_open[s])
    eeg_open[series[i]] = new_list
print(eeg_open.keys())
print(eeg_open.values())

dict_keys([2, 3, 4, 5, 7, 8, 9])
dict_values([[array([[-258., -286., -294., ...,  144.,  174.,  181.],
       [-199., -217., -212., ...,  -25.,  -13.,  -36.],
       [  40.,    2.,   15., ...,  354.,  356.,  351.],
       ...,
       [ 284.,  224.,  208., ...,  154.,  105.,  116.],
       [ 325.,  282.,  246., ...,  158.,   61.,   93.],
       [ 371.,  269.,  187., ...,  262.,   24.,   58.]], dtype=float32), array([[  288.,   307.,   323., ..., -1487., -1474., -1487.],
       [  307.,   311.,   301., ..., -1073., -1055., -1028.],
       [  300.,   306.,   303., ...,  -193.,  -225.,  -270.],
       ...,
       [ -164.,  -136.,  -161., ...,   585.,   638.,   600.],
       [ -107.,  -105.,  -126., ...,   639.,   704.,   672.],
       [  -66.,   -52.,    -2., ...,   605.,   755.,   687.]],
      dtype=float32), array([[ 320.,  305.,  295., ..., -713., -698., -714.],
       [ 272.,  279.,  292., ..., -500., -541., -551.],
       [ 266.,  242.,  261., ...,  -87.,  -44.,  -91.],
       ...,
 

In [38]:
series = [2, 3, 4, 5, 7, 8, 9]
emg_close = {}
counter = 0
for i in range (0, len(series)):
    new_list  = []
    for s in range(34*(i-1), 34*i):
        new_list.append(emg_grasp_closed[s])
    emg_close[series[i]] = new_list
print(emg_close.keys())
print(emg_close.values())

dict_keys([2, 3, 4, 5, 7, 8, 9])
dict_values([[array([[-0.03455856, -0.03785856, -0.03085856, ..., -0.06045856,
        -0.05865856, -0.05225856],
       [ 0.03487495,  0.02342495,  0.00755495, ..., -0.03993505,
        -0.03973505, -0.04003505],
       [ 0.02162418,  0.02177418,  0.02177418, ...,  0.00289418,
         0.00819418,  0.00713418],
       [ 0.04134008,  0.04393008,  0.04073008, ..., -0.01952992,
        -0.03482992, -0.03512992],
       [ 0.01030871,  0.01137871,  0.01717871, ...,  0.05623871,
         0.06707871,  0.07256871]]), array([[-0.02615856, -0.02035856, -0.01165856, ..., -0.04125856,
        -0.02875856, -0.01895856],
       [ 0.00832495,  0.00725495,  0.01045495, ..., -0.06053505,
        -0.06583505, -0.06413505],
       [-0.00540582,  0.00039418,  0.00590918, ..., -0.01590582,
        -0.00490582, -0.00050582],
       [-0.01082992,  0.00057008,  0.01052008, ...,  0.02196008,
         0.02898008,  0.04561008],
       [-0.01168129, -0.00708129, -0.00618129, ...,

In [41]:
series = [2, 3, 4, 5, 7, 8, 9]
emg_open = {}
counter = 0
for i in range (0, len(series)):
    new_list  = []
    for s in range(34*(i-1), 34*i):
        new_list.append(emg_grasp_open[s])
    emg_open[series[i]] = new_list
print(emg_open.keys())
print(emg_open.values())

dict_keys([2, 3, 4, 5, 7, 8, 9])
dict_values([[array([[ 0.01323144,  0.00774144,  0.00224144, ..., -0.00645856,
        -0.00525856, -0.00065856],
       [-0.01763505, -0.01763505, -0.01963505, ...,  0.00481495,
         0.00481495,  0.00496495],
       [-0.01320582, -0.01770582, -0.01900582, ..., -0.00540582,
        -0.00230582, -0.00460582],
       [-0.01482992, -0.01202992, -0.00992992, ..., -0.01202992,
        -0.01162992, -0.00232992],
       [-0.02328129, -0.02248129, -0.01948129, ...,  0.00131871,
         0.00466871,  0.00081871]]), array([[ 0.00194144,  0.00713144,  0.00804144, ..., -0.00645856,
        -0.00525856, -0.00065856],
       [ 0.03273495,  0.03304495,  0.03288495, ...,  0.00481495,
         0.00481495,  0.00496495],
       [-0.00630582, -0.00660582, -0.01190582, ..., -0.00540582,
        -0.00230582, -0.00460582],
       [-0.00112992, -0.00642992, -0.00622992, ..., -0.01202992,
        -0.01162992, -0.00232992],
       [-0.00068129,  0.00481871,  0.00497071, ...,